# Using Together AI with Arcade.dev to Send Emails

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Tools_and_Integrations/Agents_Arcade.ipynb)

<div style="text-align: center">
<img src="../images/Arcade.png" width="400">
</div>

#### This notebook demonstrates how to integrate Together AI's language models with Arcade's tools to create an AI agent that can send emails. We'll show:

1. Setting up the required packages and authentication
2. Configuring the email tool with Arcade
3. Creating a basic email-sending agent
4. Executing and handling the email responses

Let's see how we can combine these powerful tools to automate email communication!

## Prerequisites
- Together AI API key - see here https://api.together.ai/
- Arcade API key - see here https://arcade.dev/
- Gmail account to connect via OAuth


In [1]:
# install the required packages
!pip install -qU together arcadepy

## Gmail Configuration
We need to connect our Gmail account to Arcade. This will open a browser window for OAuth authentication.

In [2]:
import os
from arcadepy import Arcade
from together import Together

In [3]:
# Set environment variables
os.environ["TOGETHER_API_KEY"] = "XXXXXXXXXXXXX"  # Replace with your actual Together API key
os.environ["ARCADE_API_KEY"] = "arc_XXXXXXXXXXX"    # Replace with your actual Arcade API key

In [4]:

# Initialize clients
together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
arcade_client = Arcade()  # Automatically finds the ARCADE_API_KEY env variable

# Set up user ID (your email)
USER_ID = "your_email@example.com"  # Change this to your email

# Authorize Gmail access
auth_response = arcade_client.tools.authorize(
    tool_name="Google.SendEmail",
    user_id=USER_ID,
)

if auth_response.status != "completed":
    print(f"Click this link to authorize: {auth_response.url}")
    # Wait for the authorization to complete
    arcade_client.auth.wait_for_completion(auth_response)

print("Authorization completed!")

Click this link to authorize: https://accounts.google.com/o/oauth2/v2/auth?access_type=offline&client_id=826965503-pci56f36oeot675h17vic7rsc370til4.apps.googleusercontent.com&include_granted_scopes=true&prompt=consent&redirect_uri=https%3A%2F%2Fcloud.arcade.dev%2Fapi%2Fv1%2Foauth%2Fcallback&response_type=code&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=d74c9f9e-0af5-4b2c-9aba-133846b13bad
Authorization completed!


## Basic Email Tool Call

Here we create a simple integration between Together AI's LLM and Arcade's Gmail tool:
1. Initialize Together AI client
2. Get LLM to generate email parameters
3. Use Arcade to send the email

In [5]:
import json

# Define the tool schema for the LLM
email_tool_schema = {
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Send an email to a recipient",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "Email address of the recipient"
                },
                "subject": {
                    "type": "string",
                    "description": "Subject line of the email"
                },
                "body": {
                    "type": "string",
                    "description": "Body content of the email"
                }
            },
            "required": ["recipient", "subject", "body"]
        }
    }
}

# Create a chat completion with email capability
response = together_client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[
        {"role": "system", "content": "You are Alex, a product manager at an AI company."},
        {
            "role": "user",
            "content": "Send an email to John at john@arcade.dev to inquire about a good time to meet next week to review the upcoming launch.",
        },
    ],
    tools=[email_tool_schema],
)

# Extract the tool call
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    email_params = json.loads(tool_call.function.arguments)
    
    # Send the email using Arcade
    email_response = arcade_client.tools.execute(
        tool_name="Google.SendEmail",
        input={
            "recipient": email_params["recipient"],
            "subject": email_params["subject"],
            "body": email_params["body"]
        },
        user_id=USER_ID,
    )
    
    print("Email sent successfully!")
    print(f"Response: {email_response}")

Email sent successfully!
Response: ExecuteToolResponse(id='te_2y91Zi6yjeil8J4Trdt97gdvIJO', duration=313.6105537414551, execution_id='tc_2y91ZkU5ufJSWo7Z7WnqHq1ha9E', execution_type='immediate', finished_at='2025-06-06T17:47:13Z', output=Output(authorization=None, error=None, logs=None, value={'body': '', 'cc': '', 'date': '', 'from': '', 'header_message_id': '', 'history_id': '', 'id': '197465a9ffb11b15', 'in_reply_to': '', 'label_ids': ['SENT'], 'references': '', 'reply_to': '', 'snippet': '', 'subject': '', 'thread_id': '197465a9ffb11b15', 'to': '', 'url': 'https://mail.google.com/mail/u/0/#sent/197465a9ffb11b15'}), run_at=None, status='success', success=True)


## Creating a Reusable Email Agent

Let's wrap this functionality into a class for easier reuse:

In [6]:
class EmailAgent:
    def __init__(self, arcade_client, together_client, user_id):
        self.arcade = arcade_client
        self.together = together_client
        self.user_id = user_id
        self.model = "Qwen/Qwen2.5-72B-Instruct-Turbo"
        
        # Define tool schemas
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "send_email",
                    "description": "Send an email to a recipient",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "recipient": {"type": "string", "description": "Email address"},
                            "subject": {"type": "string", "description": "Email subject"},
                            "body": {"type": "string", "description": "Email body"}
                        },
                        "required": ["recipient", "subject", "body"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "list_emails",
                    "description": "List recent emails",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "n_emails": {"type": "integer", "description": "Number of emails to list"}
                        },
                        "required": ["n_emails"]
                    }
                }
            }
        ]
    
    def process_request(self, request: str, system_prompt: str = None):
        """Process a natural language request about emails."""
        
        if not system_prompt:
            system_prompt = "You are a helpful email assistant."
        
        # Get LLM response
        response = self.together.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": request}
            ],
            tools=self.tools,
        )
        
        # Handle tool calls
        if response.choices[0].message.tool_calls:
            results = []
            for tool_call in response.choices[0].message.tool_calls:
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                
                if function_name == "send_email":
                    result = self.arcade.tools.execute(
                        tool_name="Google.SendEmail",
                        input=arguments,
                        user_id=self.user_id,
                    )
                    results.append({"action": "email_sent", "result": result})
                    
                elif function_name == "list_emails":
                    result = self.arcade.tools.execute(
                        tool_name="Google.ListEmails",
                        input=arguments,
                        user_id=self.user_id,
                    )
                    results.append({"action": "emails_listed", "result": result})
            
            return results
        
        return response.choices[0].message.content

# Create the email agent
email_agent = EmailAgent(arcade_client, together_client, USER_ID)

## Example Usage

In [7]:
# Send a follow-up email
result = email_agent.process_request(
    "Send a follow-up email to sarah@design.com about the mockups for the new feature. Be friendly and ask if she needs any clarification.",
    system_prompt="You are Alex, a product manager who values clear communication."
)

print(f"Result: {result}")

Result: [{'action': 'email_sent', 'result': ExecuteToolResponse(id='te_2y91epTLLX6o29DOiN9Gef572OP', duration=337.59427070617676, execution_id='tc_2y91epbl1uQoUTih65KZEKQBasW', execution_type='immediate', finished_at='2025-06-06T17:47:53Z', output=Output(authorization=None, error=None, logs=None, value={'body': '', 'cc': '', 'date': '', 'from': '', 'header_message_id': '', 'history_id': '', 'id': '197465b38c195184', 'in_reply_to': '', 'label_ids': ['SENT'], 'references': '', 'reply_to': '', 'snippet': '', 'subject': '', 'thread_id': '197465b38c195184', 'to': '', 'url': 'https://mail.google.com/mail/u/0/#sent/197465b38c195184'}), run_at=None, status='success', success=True)}]


In [8]:
# List recent emails
emails = email_agent.process_request(
    "Show me my last 5 emails"
)

print(f"Emails: {emails}")

Emails: [{'action': 'emails_listed', 'result': ExecuteToolResponse(id='te_2y91f4GWQ4jQHKsWOLRfByN6D80', duration=814.2640590667725, execution_id='tc_2y91f3elLwIijPSce8g2l8ZzXCn', execution_type='immediate', finished_at='2025-06-06T17:47:56Z', output=Output(authorization=None, error=None, logs=None, value={'emails': [{'body': "Hi Sarah, I hope you're doing well. I just wanted to follow up on the mockups for the new feature we discussed. Please let me know if you need any further clarification or if there's anything specific you'd like to go over. Best regards, Alex", 'cc': '', 'date': 'Fri, 6 Jun 2025 12:47:52 -0500', 'from': 'shub.arcade@gmail.com', 'header_message_id': '<CAOwS8o3oV+o9-wpE1ExmxzfRxQLdWhCyw-DuZ02KPryNCiuLTA@mail.gmail.com>', 'history_id': '6305', 'id': '197465b38c195184', 'in_reply_to': '', 'label_ids': ['SENT'], 'references': '', 'reply_to': '', 'snippet': 'Hi Sarah, I hope you&#39;re doing well. I just wanted to follow up on the mockups for the new feature we discusse

## Advanced: Email Search and Analysis

Let's extend our agent to search through emails:

In [14]:
# Search for emails from a specific sender
search_result = arcade_client.tools.execute(
    tool_name="Google.ListEmailsByHeader",
    input={
        "sender": "shub@arcade.dev",
        "limit": 10
    },
    user_id=USER_ID,
)

print(f"Search result: {search_result}")
print(f"Search result output: {search_result.output}")

# Check if the search was successful and has emails
if search_result.success and search_result.output and search_result.output.value:
    emails_data = search_result.output.value
    if isinstance(emails_data, dict) and emails_data.get('emails'):
        print(f"Found {len(emails_data['emails'])} emails from John:")
        for i, email in enumerate(emails_data['emails'][:3], 1):
            print(f"{i}. Subject: {email.get('subject', 'No subject')}")
            print(f"   Snippet: {email.get('snippet', 'No preview')}")
            print()
    else:
        print("No emails found or unexpected response format")
else:
    print(f"Error searching emails: {search_result}")

Search result: ExecuteToolResponse(id='te_2y92RCYsbADfGT50fuR48BuVkt6', duration=307.7876567840576, execution_id='tc_2y92RAHqHFMqPpMBIiRERxCZ8Cl', execution_type='immediate', finished_at='2025-06-06T17:54:18Z', output=Output(authorization=None, error=None, logs=None, value={'emails': [{'body': 'Hi TogetherAI and Arcade.dev', 'cc': '', 'date': 'Fri, 6 Jun 2025 13:46:49 -0400', 'from': 'Shub Argha <shub@arcade.dev>', 'header_message_id': '<CALw=ZjmZfUc3Ebbts3ucsNPMuXP_UXZeVgEY71NzznLZPWHWSg@mail.gmail.com>', 'history_id': '6088', 'id': '197465a747b57d64', 'in_reply_to': '', 'label_ids': ['UNREAD', 'IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'], 'references': '', 'reply_to': '', 'snippet': 'Hi TogetherAI and Arcade.dev', 'subject': 'Hi Together!', 'thread_id': '197465a747b57d64', 'to': 'shub.arcade@gmail.com'}]}), run_at=None, status='success', success=True)
Search result output: Output(authorization=None, error=None, logs=None, value={'emails': [{'body': 'Hi TogetherAI and Arcade.dev', 'cc':

## Summary

You've successfully created an email agent using Arcade and Together AI! This notebook demonstrated:

1. **OAuth Authentication** - Securely connecting your Gmail account to Arcade
2. **Tool Integration** - Using Together AI to generate email parameters and Arcade to execute actions
3. **Reusable Agent** - Building a flexible EmailAgent class
4. **Multiple Actions** - Sending emails, listing emails, and searching by headers

### Next Steps
- Explore more Arcade tools like Calendar, Slack, and Drive
- Build complex workflows combining multiple tools
- Add error handling and retry logic for production use

Check out the [Arcade documentation](https://docs.arcade.dev/) for more tools and capabilities!